# Criterion for convergence in backward error after fault

In [1]:
%matplotlib inline

In [2]:
import numpy as np
from sys import stdout
import math


## Import modules

In [3]:
from Experiment import *
from Parameters import *
from Fault import *
from Algorithm import *
from Utils import *

## Algorithm initialization

In [4]:
import gmres
import gmres_display

### algorithm parameters

In [5]:
# parameters that won't change during the whole experiment
epsilon = 1.e-10

algorithm_parameters = Parameters()
algorithm_parameters["orthMethod"] = gmres.modified_gramschmidt
algorithm_parameters["dtype"] = 'd'
algorithm_parameters["vulnerable"] = True
algorithm_parameters["save_data"] = {"iteration_count",
                                     "residual",
                                     "residuals",
                                     "true_residual",
                                     "true_residuals",
                                     "orthogonality",
                                     #"arnoldi",
                                     "faults",
                                     "H",
                                     "V",
                                     "y",
                                     "checksum",
                                     "threshold",
                                     "criteria",
                                     "delta"
                                    }

AttributeError: 'module' object has no attribute 'modified_gramschmidt'

### implementation

In [ ]:
def implementation(self, input, algorithm_parameters, experiment_parameters):
    gmres.implementation(self, input, algorithm_parameters, experiment_parameters)

In [ ]:
gmres_algorithm = Algorithm(algorithm_parameters, implementation)

## Experiment setup

## Load input

In [ ]:
input = load_mat("./gre_216a.mat") #load_mat("./gre_216a.mat")
A = input["A"]
inputs = [input]
n = A.shape[0]

In [ ]:
plt.matshow(A)

## Experiment initialization

### Experiment parameters

In [ ]:
# Parameters that may change during the experiment
# Experiment parameters override Algorithm parameters
experiment_parameters = Parameters()

experiment_parameters["vulnerable"] = False
experiment_parameters["tol"] = epsilon
experiment_parameters["full"] = False

In [ ]:
F = Experiment(experiment_parameters, gmres_algorithm)
F.set_inputs(inputs)

## Run experiment

In [ ]:
F.run(show_progress=True, n=1)

In [ ]:
minimum = min(filter(lambda d: d[1] < epsilon, enumerate(F.get_data("true_residuals")[0])),
                    key=lambda d: d[0])
print minimum
min_iteration = minimum[0]

In [ ]:
gmres_display.convergence_history(F.get_data()[0], ylim = (epsilon/10, 1))

## Effect of faults in GMRES

### Experiment parameters

In [ ]:
# Parameters that may change during the experiment
experiment_parameters = Parameters()

experiment_parameters["vulnerable"] = True
fault_parameters = {}
fault_parameters["fault_callback"] = lambda fault: (stdout.write(" %s " % str(fault)))
fault_parameters["register"] = "right"
fault_parameters["vulnerable"] = True
fault_parameters["vulnerable_bits"] = [int(np.random.rand() * 64)]
fault_parameters["fault_indices"] = {"i": int(np.random.rand()*n), 
                                     "j":0, 
                                     "k":int(np.random.rand()*n)}
fault_parameters["timer"] = int(np.random.rand() * min_iteration)
experiment_parameters["fault_parameters"] = fault_parameters


#### Implementation (gmres)

In [ ]:
def implementation(self, input, algorithm_parameters, experiment_parameters):
    gmres.implementation(self, input, algorithm_parameters, experiment_parameters)

In [ ]:
gmres_algorithm = Algorithm(algorithm_parameters, implementation)

In [ ]:
E = Experiment(experiment_parameters, gmres_algorithm)
E.set_inputs(inputs)
E.run(show_progress=True, n=1)

In [ ]:
gmres_display.convergence_history(E.get_data()[0], data_no_fault = F.get_data()[0], fault = True, delta = False,  xlim = (0, 200), ylim = (epsilon/10, 1),xytext = (5, epsilon / 10), bbox_to_anchor=(0.5, 0.85))

### Parameters influence

In [ ]:
def experiment_parameters_generator(vulnerable = True,
                                    tol = epsilon,
                                    bits = [40], 
                                    iterations = [min_iteration/2], 
                                    registers = ["left"], 
                                    indices = [{"i":29, "j":0, "k":56}],
                                    fault_callback = lambda fault: (stdout.write(" %s " % str(fault)))
                                   ):
    parameters = Parameters()
    parameters["vulnerable"] = vulnerable
    parameters["tol"] = tol
    fault_parameters = {}
    fault_parameters["fault_callback"] = fault_callback
    fault_parameters["vulnerable"] = vulnerable
    while (True):

        parameters["fault_parameters"] = fault_parameters

        for bit in bits:
            for iteration in iterations:
                for reg in registers:
                    for indice in indices:
                        fault_parameters["vulnerable_bits"] = [bit]
                        fault_parameters["timer"] = iteration
                        fault_parameters["register"] = reg
                        fault_parameters["fault_indices"] = indice
                        yield parameters

#### Bit flipped

In [ ]:


experiment_parameters = experiment_parameters_generator(bits = [60, 40, 20])

E = Experiment(experiment_parameters, gmres_algorithm)
E.set_inputs(inputs)
E.run(show_progress=True, n=3)

In [ ]:
for data in E.get_data():
    gmres_display.convergence_history(data, data_no_fault = F.get_data()[0], fault = True, delta = False, ylim = (epsilon/10, 1), xytext = (5, epsilon/10), bbox_to_anchor=(0.5, 0.8))
    plt.show()

#### Iteration

In [ ]:
experiment_parameters = experiment_parameters_generator(iterations = [0, min_iteration / 3, (2 * min_iteration) / 3])

E = Experiment(experiment_parameters, gmres_algorithm)
E.set_inputs(inputs)
E.run(show_progress=True, n=3)

In [ ]:
for data in E.get_data():
    gmres_display.convergence_history(data, data_no_fault = F.get_data()[0], fault = True, delta = False, ylim = (1.e-16, 1), xytext = (5, 1.e-16), bbox_to_anchor=(0.6, 0.8))
    plt.show()

#### register

In [ ]:
experiment_parameters = experiment_parameters_generator(registers = ["left", "right", "middle"])

E = Experiment(experiment_parameters, gmres_algorithm)
E.set_inputs(inputs)
E.run(show_progress=True, n=3)

In [ ]:
for data in E.get_data():
    gmres_display.convergence_history(data, data_no_fault = F.get_data()[0], fault = True, delta = False, xlim = (0, 200), ylim = (1.e-16, 1), xytext = (5, 1.e-16), bbox_to_anchor=(0.6, 0.8))
    plt.show()

#### location

In [ ]:
experiment_parameters = experiment_parameters_generator(indices = [{"i": int(np.random.rand()*n), 
                                                                    "j":0, 
                                                                    "k":int(np.random.rand()*n)} for i in xrange(3)]  
)

E = Experiment(experiment_parameters, gmres_algorithm)
E.set_inputs(inputs)
E.run(show_progress=True, n=3)

In [ ]:
for data in E.get_data():
    gmres_display.convergence_history(data, data_no_fault = F.get_data()[0], fault = True, delta = False, xlim = (0, 200), ylim = (1.e-16, 1), xytext = (5, 1.e-16), bbox_to_anchor=(0.6, 0.8))
    plt.show()

## Quantitative study

In [ ]:
step_ite = 4
step_bit = 3


experiment_parameters = experiment_parameters_generator(bits = np.arange(63, -1, -step_bit),
                                                       iterations = np.arange(0, min_iteration, step_ite))

E = Experiment(experiment_parameters, gmres_algorithm)
E.set_inputs(inputs)
E.run(show_progress=True, n=int(math.ceil(float(min_iteration)/step_ite)) * int(math.ceil(64./step_bit)))

In [ ]:
def has_converged(data, epsilon = 1.e-12):
    return len(filter (lambda d: d[0] < epsilon and d[1] < epsilon, zip(data["true_residuals"], data["residuals"]))) > 0

def when_has_converged(data, epsilon = 1.e-12):
    iteration_residual = filter (lambda d: d[1][0] < epsilon and d[1][1] < epsilon, 
                                 enumerate(zip(data["true_residuals"], data["residuals"])))
    if iteration_residual:
        return min(iteration_residual, key=lambda d:d[0])[0]
    else:
        return None

In [ ]:

    
min_iteration = when_has_converged(F.get_data()[0], epsilon = epsilon)

def scatter_bit_injection(data, min_iteration = None,
                          convergence = None, delay = None, 
                          color = 'r', marker = 'o', area = 75, alpha = 0.5,
                          xlabel = "Flipped bit", ylabel="Fault injection time (%)",
                          xlim = [-5, 64], ylim = [-5, 105],
                          epsilon = 1.e-12):
    if convergence != None:
        if convergence:
            data = filter(lambda d: has_converged(d, epsilon), data)
            if delay != None :
                if not min_iteration:
                        print "min_iteration should be set"
                        return
                if delay:
                    data = filter(lambda d: when_has_converged(d, epsilon) > min_iteration, data)
                else:
                    data = filter(lambda d: when_has_converged(d, epsilon) <= min_iteration, data)
        else:
            data = filter(lambda d: not has_converged(d, epsilon), data)
    
    X = map(lambda f: f["faults"][0]["bit"], data)
    if min_iteration:
        Y = map(lambda f: (100*f["faults"][0]["timer"]) / min_iteration, data)
    else:
        Y = map(lambda f: (f["faults"][0]["timer"]), data)
    C = []
    
    if hasattr(color, '__call__'):
        C = map(color, data)
    
    S = []
    if hasattr(area, '__call__'):
        S = map(area, data)
    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    if xlim:
        plt.xlim(xlim)
    if ylim:
        plt.ylim(ylim)

    return plt.scatter(X, Y, marker=marker, c=C, s=S, alpha = alpha)


scatter_bit_injection(E.get_data(), 
                      color = lambda d: when_has_converged(d, epsilon) if has_converged(d, epsilon) else 216,
                      area = lambda d: 75,
                      ylabel = "Fault injection time (iteration)",
                      ylim = [-5, min_iteration+5])
plt.colorbar()
plt.show()

converged = scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = True,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75)

delayed = scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = True, delay = True,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75)

not_converged = scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = False,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75)

plt.legend((converged, delayed, not_converged),
           ('Converged', 'Converged (delayed)', 'Not converged'))

plt.show()


plt.subplot(2, 1, 1)
scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = False,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75,
                      xlabel = None, ylabel = "not converged")

plt.subplot(2, 1, 2)
scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = True,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75,
                      xlabel = None, ylabel = "converged")

   
#plt.xlabel("flipped bit")
#plt.ylabel("Fault injection time (%)")       
    
#scatter_bit_injection(E.get_data(), min_iteration)

#plt.legend((conv, conv_delay, no_conv),
#           ('Convergence', 'Convergence (no delay)', 'No convergence'),
#           scatterpoints=1,
#           loc='upper right',
#           ncol=1,
#           fontsize=8)




In [ ]:
scatter_bit_injection(E.get_data(), 
                      color = lambda d: when_has_converged(d, epsilon) if has_converged(d, epsilon) else n,
                      area = lambda d: 75,
                      ylabel = "Fault injection time (iteration)",
                      ylim = [-5, min_iteration+5])
plt.colorbar()
plt.show()

converged = scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = True,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75)

delayed = scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = True, delay = True,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75)

not_converged = scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = False,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75)

plt.legend((converged, delayed, not_converged),
           ('Converged', 'Converged (delayed)', 'Not converged'))

plt.show()


plt.subplot(2, 1, 1)
scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = False,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75,
                      xlabel = None, ylabel = "not converged")

plt.subplot(2, 1, 2)
scatter_bit_injection(E.get_data(), min_iteration, 
                      convergence = True,
                      color = lambda d: "red" if not has_converged(d, epsilon) else "blue" if when_has_converged(d, epsilon) > min_iteration else "green",
                      area = lambda d: 75,
                      xlabel = None, ylabel = "not converged")

## Quantitative results

In [ ]:
def experiment_parameters_generator_random(vulnerable = True,
                                    tol = 0,
                                    m = 100,
                                    fault_callback = lambda fault: (stdout.write(" %s " % str(fault)))
                                   ):
    import random
    
    parameters = Parameters()
    parameters["vulnerable"] = vulnerable
    parameters["tol"] = tol
    fault_parameters = {}
    fault_parameters["fault_callback"] = fault_callback
    fault_parameters["vulnerable"] = vulnerable
            
    parameters["fault_parameters"] = fault_parameters
        
    while (True):
        fault_parameters["timer"] = random.choice([i for i in xrange(m)])
        yield parameters

## Fault detection

### Qualitative results

In [ ]:
input = load_mat("./gre_216a.mat")
A = input["A"]
inputs = [input]
n = A.shape[0]

In [ ]:
experiment_parameters = experiment_parameters_generator(bits = [60, 50, 40], tol = epsilon)

E_ = Experiment(experiment_parameters, gmres_algorithm)
E_.set_inputs(inputs)
E_.run(show_progress=True, n=3)

In [ ]:
for data in E_.get_data():
    gmres_display.convergence_history(data, data_no_fault = F.get_data()[0], fault = True, delta = False, checksum=True, xlim = (0, 200), ylim = (1.e-16, 1), xytext = (5, 1.e-16), bbox_to_anchor=(0.6, 0.8))
    plt.show()

In [ ]:
for data in E_.get_data():
    gmres_display.convergence_history(data, data_no_fault = F.get_data()[0], c = 0.5, fault = True, delta = False, checksum=True, threshold = True, xlim = (0, 200), ylim = (1.e-16, 1), xytext = (5, 1.e-16), bbox_to_anchor=(0.6, 0.8))
    plt.show()

In [ ]:
def criterion(data, c = 0.5, epsilon = 1.e-12):
    m = when_has_converged(data, epsilon = (1-c)*epsilon)
    if len(data["checksum"]) >= m and len(data["threshold"]) >= m:
        return reduce(lambda d, e: d and (e[0] < c * e[1]), zip(data["checksum"][:m], data["threshold"][:m])) and data["checksum"][0] < c * data["threshold"][0]
    else:
        return True

In [ ]:
def false_detection(data, c = 0.5, epsilon = 1.e-12, key_checksum="checksum", key_threshold="threshold"):
    m = when_has_converged(data, epsilon = (1-c)*epsilon)
    for i, (checksum, threshold) in enumerate(zip(data[key_checksum][:m], data[key_threshold][:m])):
        if checksum > c * threshold:
            if data["faults"][0]["timer"] != i:
                return True
    return False

def no_impact_fault_detection(data, c = 0.5, epsilon = 1.e-12, key_checksum="checksum", key_threshold="threshold"):
    m = when_has_converged(data, epsilon = (1-c)*epsilon)
    f = data["faults"][0]["timer"]
    return (has_converged(data, epsilon = epsilon) and 
            data[key_checksum][f] > c * data[key_threshold][f])
        
def no_impact_fault_no_detection(data, c = 0.5, epsilon = 1.e-12, key_checksum="checksum", key_threshold="threshold"):
    f = data["faults"][0]["timer"]
    return (has_converged(data, epsilon = epsilon) and
            data[key_checksum][f] < c * data[key_threshold][f])
        
def fault_no_detection(data, c = 0.5, epsilon = 1.e-12, key_checksum="checksum", key_threshold="threshold"):
    f = data["faults"][0]["timer"]
    return (not has_converged(data, epsilon = epsilon)  and
            data[key_checksum][f] < c * data[key_threshold][f])
        
def fault_detection(data, c = 0.5, epsilon = 1.e-12, key_checksum="checksum", key_threshold="threshold"):
    f = data["faults"][0]["timer"]
    return (not has_converged(data, epsilon = epsilon) and
            data[key_checksum][f] > c * data[key_threshold][f])
               

In [ ]:
print criterion(E_.get_data()[0])
print criterion(E_.get_data()[1])
print criterion(E_.get_data()[2])

In [ ]:
for data, result in zip(E.get_data(), E.get_results()):
    for i in xrange(len(data["threshold"])):
        data["threshold"][i] = (np.linalg.norm(result[0]["b"]) * epsilon) / abs(data["y"][i][i][0])

In [ ]:
for data in E.get_data():
    data["true_threshold"] = []
for data, result in zip(E.get_data(), E.get_results()):
    for i in xrange(len(data["threshold"])):
        data["true_threshold"] += [(np.linalg.norm(result[0]["b"]) * epsilon) / abs(data["y"][-1][i][0])]

In [ ]:
for data in E.get_data():
    data["true_checksum"] = []
for data, result in zip(E.get_data(), E.get_results()):
    for i in xrange(len(data["checksum"])):
        data["true_checksum"] += [0]
    data["true_checksum"][data["faults"][0]["timer"]] = abs(data["faults"][0]["value_before"]- data["faults"][0]["value_after"])

In [ ]:
data = filter(lambda d: len(d["faults"]) > 0, E.get_data())
epsilon = 1.e-6

for d in filter(lambda d: no_impact_fault_detection(d, epsilon = epsilon, c=0.5, key_threshold="true_threshold", key_checksum="true_checksum"), data)[0:10]:
    gmres_display.convergence_history(d, data_no_fault = F.get_data()[0], c = 0.5, fault = True, delta = False, checksum=True, threshold = True, xlim = (0, int(1.5 * min_iteration)), ylim = (epsilon / 10, 1), xytext = (1, epsilon/10), bbox_to_anchor=(0.6, 0.8))
    plt.show()

In [ ]:

X = [1.2**(-50+i) for i in xrange(51)]
test_orange = []
test_red = []
test_green = []
test_gray1 = []
test_gray2 = []


for x in X:
    #test_red += [(100. * len(filter_data(data, {'converged':False, 'criterion':True}, c=x)))/len(data)]
    #test_green += [(100. * len(filter_data(data, {'converged':False, 'criterion':False}, c=x)))/len(data)]
    #test_gray1 += [(100. * len(filter_data(data, {'converged':True, 'criterion':True}, c=x)))/len(data)]
    #test_gray2 += [(100. * len(filter_data(data, {'converged':True, 'criterion':False}, c=x)))/len(data)]

    test_orange += [(100. * len(filter(lambda d: false_detection(d, epsilon = epsilon, c=x, key_threshold="true_threshold", key_checksum="true_checksum"), data)))/len(data)]
    test_red += [(100. * len(filter(lambda d: fault_no_detection(d, epsilon = epsilon, c=x, key_threshold="true_threshold", key_checksum="true_checksum"), data)))/len(data)]
    test_green += [(100. * len(filter(lambda d: fault_detection(d, epsilon = epsilon, c=x, key_threshold="true_threshold", key_checksum="true_checksum"), data)))/len(data)]
    test_gray1 += [(100. * len(filter(lambda d: no_impact_fault_no_detection(d, epsilon = epsilon, c=x, key_threshold="true_threshold", key_checksum="true_checksum"), data)))/len(data)]
    test_gray2 += [(100. * len(filter(lambda d: no_impact_fault_detection(d, epsilon = epsilon, c=x, key_threshold="true_threshold", key_checksum="true_checksum"), data)))/len(data)]
    

Y = np.row_stack((test_gray1, test_gray2, test_green, test_red))   
Y = np.cumsum(Y, axis=0)  

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(X, test_orange, c="orange", linestyle="--")
ax.fill_between(X, 0, Y[0,:], facecolor="lightgray", alpha=.7, label="No impact fault ignored")
ax.fill_between(X, Y[0,:], Y[1,:], facecolor="darkgray", alpha=.7, label="No impact fault detected")
ax.fill_between(X, Y[1,:], Y[2,:], facecolor="green", alpha=.7, label="Critical fault detected")
ax.fill_between(X, Y[2,:], Y[3,:], facecolor="red",alpha=.7, label="Critical fault ignored")
ax.set_xscale('log')
ax.set_ylim([0, 100])
ax.set_xlim([0.0001, 1])
ax.set_xlabel("c")
ax.set_ylabel("%")
ax.set_title("Test result repartition")

p1 = plt.Rectangle((0, 0), 1, 1, fc="green", alpha=.7)
p2 = plt.Rectangle((0, 0), 1, 1, fc="lightgray", alpha=.7)
p3 = plt.Rectangle((0, 0), 1, 1, fc="darkgray", alpha=.7)
p4 = plt.Rectangle((0, 0), 1, 1, fc="red", alpha=.7)
plt.legend([p1, p2, p3, p4], ["Critical fault detected", "No impact fault ignored", "No impact fault detected", "Critical fault ignored"], loc=4)
plt.plot([0.5, 0.5], [0, 100], c = "red", linestyle = "--")
plt.show()

In [ ]:
c = 0.5

In [ ]:
def show_test_result (data, no_fault_data, c = 0.5, epsilon = 1.e-12, key_checksum="true_checksum", key_threshold="true_threshold"):

    min_iteration = when_has_converged(no_fault_data, epsilon = epsilon)

    
    false_detection_plot = scatter_bit_injection(filter(lambda d: false_detection(d, c=c, epsilon=epsilon, key_checksum="true_checksum", key_threshold="true_threshold"), data), 
                                            min_iteration, 
                                            color = lambda d:"orange",
                                            area = lambda d: 30,
                                            alpha = 1)

    fault_no_detection_plot = scatter_bit_injection(filter(lambda d: fault_no_detection(d, c=c, epsilon=epsilon, key_checksum="true_checksum", key_threshold="true_threshold"), data), 
                                            min_iteration, 
                                            color = lambda d:"red",
                                            area = lambda d: 100)

    fault_detection_plot = scatter_bit_injection(filter(lambda d: fault_detection(d, c=c, epsilon=epsilon, key_checksum="true_checksum", key_threshold="true_threshold"), data), 
                                            min_iteration, 
                                            color = lambda d:"green",
                                            area = lambda d: 100)

    no_impact_fault_no_detection_plot = scatter_bit_injection(filter(lambda d: no_impact_fault_no_detection(d, c=c, epsilon=epsilon, key_checksum="true_checksum", key_threshold="true_threshold"), data), 
                                            min_iteration, 
                                            color = lambda d:"#CCCCCC",
                                            area = lambda d: 100)

    no_impact_fault_detection_plot = scatter_bit_injection(filter(lambda d: no_impact_fault_detection(d, c=c, epsilon=epsilon, key_checksum="true_checksum", key_threshold="true_threshold"), data), 
                                            min_iteration, 
                                            color = lambda d:"#222222",
                                            area = lambda d: 100)



    


    plt.legend((false_detection_plot, fault_no_detection_plot, fault_detection_plot, no_impact_fault_no_detection_plot, no_impact_fault_detection_plot),
           ('False detection', 'Critical fault ignored', 'Critical fault detected', 'No impact fault ignored', 'no impact fault detected'))


show_test_result(E.get_data(), F.get_data()[0], c=c, epsilon = epsilon)    
plt.show()



In [ ]:
import os

epsilon = 1.e-12
c = 0.5

for mat_file in os.listdir(os.getcwd()):
    if mat_file.endswith(".mat"): 
        print mat_file
        _inputs = [load_mat(mat_file)]
        _F = Experiment(Parameters({"tol":epsilon}), gmres_algorithm)
        _F.set_inputs(_inputs)
        _F.run(n = 1)
        if has_converged(F.get_data()[0], epsilon = epsilon):
            _min_iteration = when_has_converged(_F.get_data()[0], epsilon = epsilon)
            if _min_iteration:
                _G = Experiment(experiment_parameters_generator_random(m=_min_iteration, tol = (1-c)*epsilon), gmres_algorithm)
                _G.set_inputs(inputs)
                _G.run(show_progress=True, n=100)
                show_test_result(_G.get_data(), _F.get_data()[0], c = c, epsilon = epsilon)


